In [ ]:
!pip install --upgrade pip
!pip install torch_geometric
!pip install torch

In [24]:
import torch 
from torch_geometric.nn import GCNConv

from torch_geometric.utils import negative_sampling

from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T

from sklearn.metrics import roc_auc_score


In [31]:
device = torch.device("cpu")

transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, 
                      num_test=0.1, 
                      is_undirected=True, 
                      add_negative_train_samples=False)
])

# data
dataset = Planetoid(root="testdata", name="Cora", transform=transform)


print("Data Info: ", dataset.name)
print("Number of graphs: ", len(dataset))
print("Number of features: ", dataset.num_features)
print("Number of classes (node types): ", dataset.num_classes)
print("Number of edge features: ", dataset.num_edge_features)
print("Number of node features: ", dataset.num_node_features )



train_data, val_data, test_data = dataset[0]

<bound method Dataset.get_summary of Cora()>
Data Info:  Cora
Number of graphs:  1
Number of features:  1433
Number of classes (node types):  7
Number of edge features:  0
Number of node features:  1433


In [17]:
# GCNN Model Implementation

class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)
    
    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)
    
    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

In [18]:
model = Net(dataset.num_features, 128, 64)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index,
        num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1),
        method='sparse'
    )

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1
    )

    edge_label = torch.cat(
        [train_data.edge_label, train_data.edge_label.new_zeros(neg_edge_index.size(1))],
        dim=0
    )

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss


In [20]:
@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())


In [23]:
best_val_auc = final_test_auc = 0
for epoch in range(1,101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc

    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, ' f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')
z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)

Epoch: 001, Loss: 0.4133, Val: 0.9399, Test: 0.9062
Epoch: 002, Loss: 0.4125, Val: 0.9402, Test: 0.9068
Epoch: 003, Loss: 0.4235, Val: 0.9405, Test: 0.9076
Epoch: 004, Loss: 0.4193, Val: 0.9401, Test: 0.9072
Epoch: 005, Loss: 0.4200, Val: 0.9404, Test: 0.9070
Epoch: 006, Loss: 0.4185, Val: 0.9400, Test: 0.9065
Epoch: 007, Loss: 0.4165, Val: 0.9398, Test: 0.9071
Epoch: 008, Loss: 0.4268, Val: 0.9396, Test: 0.9074
Epoch: 009, Loss: 0.4137, Val: 0.9395, Test: 0.9080
Epoch: 010, Loss: 0.4237, Val: 0.9399, Test: 0.9082
Epoch: 011, Loss: 0.4174, Val: 0.9407, Test: 0.9080
Epoch: 012, Loss: 0.4266, Val: 0.9400, Test: 0.9080
Epoch: 013, Loss: 0.4152, Val: 0.9395, Test: 0.9085
Epoch: 014, Loss: 0.4144, Val: 0.9400, Test: 0.9098
Epoch: 015, Loss: 0.4191, Val: 0.9403, Test: 0.9108
Epoch: 016, Loss: 0.4250, Val: 0.9407, Test: 0.9109
Epoch: 017, Loss: 0.4231, Val: 0.9409, Test: 0.9109
Epoch: 018, Loss: 0.4186, Val: 0.9408, Test: 0.9114
Epoch: 019, Loss: 0.4173, Val: 0.9402, Test: 0.9112
Epoch: 020, 

In [32]:
final_edge_index

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [   0,    2,    7,  ..., 2703, 2706, 2707]])